In [1]:
import billboard


In [2]:
# Call hot 100 tracks billbaord API using billboard library
chart = billboard.ChartData('hot-100')

In [3]:
artists = []
songs = []


for song in chart:
    
#     Add songs and artists to list for API calls
    songs.append(song.title)
    artists.append(song.artist)
# print(artists)
# print()
# print(songs)

In [4]:
# Call top 200 tracks billbaord API using billboard library
chart = billboard.ChartData('billboard-200')
# print(chart)

In [5]:
import requests
import json

top_songs_json = []
song_ids = []
popularity = []

# Enter your access_token given to you. Will import have to figure out a way to import from js
access_token = "BQBbfSL2tL2FmISsRDVilkmAlvFVqfKEwHAG3l9THmdfNj6J588lD97FGNS2-b50Kin1T19UUo8NiPivTA5m11t4lUFPnPO7pTpXzBdTgGI45syQeCs6auNzyF4VuY-Gjb95V9Re9F31O5w_DpWuCJ1AmnJvgh7fISoI"

for song in songs:
    
#     Spotify API search endpoint returns list of tracks matching track title in order of popularity
    endpoint = f"https://api.spotify.com/v1/search?q={song}&type=track"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    
    try:
#         Apped first json object to list (The most popular track matching search criteria)
        track = response["tracks"]["items"][0]
        top_songs_json.append(track)
        
#         Append song_id for use in second API Call
        song_id = response["tracks"]["items"][0]["id"]
        song_ids.append(song_id)
        
    except: 
#         Alert for any track that was not appended to lists
        print("This was not added")
        print(json.dumps(response["tracks"]["items"][0]["artists"],indent=4))


In [ ]:
# Verify list length
print(len(song_ids))

In [6]:
# raw json
audio_features = []

# List to Load into db
features = []

for song_id in song_ids:
    
#     Call Spotify API to collect Audio features data
    endpoint = f"https://api.spotify.com/v1/audio-features/{song_id}"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    
    song_id = response["id"]
    
    endpoint2 = f"https://api.spotify.com/v1/tracks/{song_id}"
    headers2 = {"Authorization":f"Bearer {access_token}"}

    response2 = requests.get(endpoint2,headers=headers2).json()
    
#     Append raw json to list
    audio_features.append(response)

    
#     Append each feature
    features.append({
        "id": response["id"],
        "popularity": response2["popularity"],
        "danceability": response["danceability"],
        "energy": response["energy"],
        "loudness": response["loudness"],
        "speechiness": response["speechiness"],
        "duration_ms": response["duration_ms"],
        "tempo": response["tempo"]
    })
#     danceability.append(response["danceability"])
#     energy.append(response["energy"])
#     loudness.append(response["loudness"])
#     speechiness.append(response["speechiness"])
#     duration_ms.append(response["duration_ms"])
#     tempo.append(response["tempo"])
    

In [7]:
print(features)

[{'id': '6FRLCMO5TUHTexlWo8ym1W', 'popularity': 94, 'danceability': 0.851, 'energy': 0.541, 'loudness': -6.825, 'speechiness': 0.0505, 'duration_ms': 235545, 'tempo': 124.959}, {'id': '0s3nnoMeVWz3989MkNQiRf', 'popularity': 97, 'danceability': 0.511, 'energy': 0.566, 'loudness': -7.23, 'speechiness': 0.2, 'duration_ms': 239836, 'tempo': 83.903}, {'id': '7dt6x5M1jzdTEt8oCbisTK', 'popularity': 95, 'danceability': 0.68, 'energy': 0.563, 'loudness': -5.843, 'speechiness': 0.0454, 'duration_ms': 231267, 'tempo': 145.028}, {'id': '1BxkZE73h9BN3qwuA15TA3', 'popularity': 91, 'danceability': 0.9, 'energy': 0.653, 'loudness': -6.962, 'speechiness': 0.289, 'duration_ms': 145603, 'tempo': 112.503}, {'id': '2VxeLyX666F8uXCJ0dZF8B', 'popularity': 84, 'danceability': 0.572, 'energy': 0.385, 'loudness': -6.362, 'speechiness': 0.0308, 'duration_ms': 215733, 'tempo': 95.799}, {'id': '2xLMifQCjDGFmkHkpNLD9h', 'popularity': 97, 'danceability': 0.834, 'energy': 0.73, 'loudness': -3.714, 'speechiness': 0.22

In [8]:
import sqlite3

conn = sqlite3.connect('music.db')

c = conn.cursor()


In [15]:
 
from sqlalchemy import create_engine

db_uri = "sqlite:///music.db"
engine = create_engine(db_uri)

# create table
# engine.execute('CREATE TABLE "features" ('
#                'id string NOT NULL,'
#                'popularity integer,'
#                'danceability integer,'
#                'energy integer,'
#                'loudness integer,'
#                'speechiness integer,'
#                'duration_ms integer,'
#                'tempo integer);')

# c.execute("DROP TABLE features")
result = engine.execute('SELECT * FROM "features"')
print(result.fetchall())

[('6FRLCMO5TUHTexlWo8ym1W', 94, 0.851, 0.541, -6.825, 0.0505, 235545, 124.959), ('0s3nnoMeVWz3989MkNQiRf', 97, 0.511, 0.566, -7.23, 0.2, 239836, 83.903), ('7dt6x5M1jzdTEt8oCbisTK', 95, 0.68, 0.563, -5.843, 0.0454, 231267, 145.028), ('1BxkZE73h9BN3qwuA15TA3', 91, 0.9, 0.653, -6.962, 0.289, 145603, 112.503), ('2VxeLyX666F8uXCJ0dZF8B', 84, 0.572, 0.385, -6.362, 0.0308, 215733, 95.799), ('2xLMifQCjDGFmkHkpNLD9h', 97, 0.834, 0.73, -3.714, 0.222, 312820, 155.008), ('55S2PQgSMYAhgoTCcGCDfw', 90, 0.596, 0.854, -5.114, 0.463, 203418, 120.274), ('2dpaYNEQHiRxtZbfNsse99', 100, 0.687, 0.792, -2.749, 0.0452, 214290, 100.015), ('58q2HKrzhC3ozto2nDdN4z', 96, 0.816, 0.726, -3.998, 0.129, 253390, 136.048), ('2E124GmJRnBJuXbTb4cPUB', 96, 0.931, 0.387, -9.127, 0.412, 179405, 125.978), ('2G7V7zsVDxg1yRsu7Ew9RJ', 99, 0.835, 0.626, -5.833, 0.125, 217925, 91.03), ('45Egmo7icyopuzJN0oMEdk', 89, 0.708, 0.648, -5.626, 0.0449, 201707, 143.955), ('4zIO3ilp5HvTeK3HJHxhMP', 95, 0.448, 0.612, -6.106, 0.0708, 189467,

In [11]:
import sqlite3

conn = sqlite3.connect('music.db')

c = conn.cursor()


for feature in features:
    c.execute("insert into features values (?, ?, ?, ?, ?, ?, ?, ?)", 
              [feature['id'],feature['popularity'],feature['danceability'],feature['energy'],
              feature['loudness'],feature['speechiness'],feature['duration_ms'],feature['tempo']])
    conn.commit()
conn.close()

ProgrammingError: Cannot operate on a closed database.